# Part 2: Get Coordinates

### Download the Dataset

In [1]:
# import pandas
import pandas as pd

In [3]:
# Read data
HTMLPAGE = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data = pd.read_html(HTMLPAGE)
df1 = data[0]

### Modify the dataset

In [5]:
# Set headers
headers = ["PostalCode","Borough","Neighborhood"]
df1.columns = headers

# Drop rows where Borough == 'Not assigned'
df = df1[df1['Borough'] != 'Not assigned'].reset_index(drop=True)

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        df.iloc[index, 2] = row['Borough']
        
#Group by postalcode and borough
grouped = df.groupby(['PostalCode', 'Borough'], as_index=False).sum()
grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Update Neighborhood by setting commas. 
def get_neighborhood(row):
    neigh = ""
    
    postalcode = row['PostalCode']
    df_neighbourhoods = df[df['PostalCode'] == postalcode].reset_index(drop=True)
    
    for index, row in df_neighbourhoods.iterrows():
        #print(row['PostalCode'], row['Borough'])
        if index > 0:
            neigh = neigh + ', ' + row['Neighborhood']
        else:
            neigh = row['Neighborhood']
            
    return neigh

grouped['Neighborhood'] = grouped.apply(get_neighborhood, axis=1)
grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
grouped.shape

(103, 3)

#### Get the geographical coordinates of each postal code

In [8]:

path = "https://cocl.us/Geospatial_data"
coordinates_df = pd.read_csv(path)
coordinates_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
# See the rownumber
coordinates_df.shape

(103, 3)

In [12]:
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join the dataframes

In [13]:
dfinal = grouped.merge(coordinates_df, on="PostalCode", how = 'inner')
dfinal.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
# Test some result:
dfinal[dfinal['Neighborhood'] == 'Central Bay Street']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


The value above is same as in the assignment given.